In [1]:
pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [2]:
import chromadb

client = chromadb.CloudClient(
  api_key='ck-7LcEbX4rQYfMVFzcQ7kEhFWLB4YjwcnPRwKfJ3heuFpK',
  tenant='61cb8b18-7560-4214-ba47-ce038db710c3',
  database='eurontextimage'
)

In [3]:
client

In [4]:
pip install pymupdf sentence-transformers chromadb pillow

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
from typing import List, Dict, Any

import fitz  # PyMuPDF
from PIL import Image
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb

d:\finetuning\fullfinetuning\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
PDF_PATH = r"attention is all you need.pdf"   # path to your PDF (same folder as script/notebook)
FIGURES_DIR = "figures"                       # folder for extracted images

os.makedirs(FIGURES_DIR, exist_ok=True)

In [8]:
client = chromadb.CloudClient(
  api_key='ck-7LcEbX4rQYfMVFzcQ7kEhFWLB4YjwcnPRwKfJ3heuFpK',
  tenant='61cb8b18-7560-4214-ba47-ce038db710c3',
  database='eurontextimage'
)

In [9]:
FIG_COLLECTION_NAME = "attention_figures"
TEXT_COLLECTION_NAME = "attention_text_chunks"

In [ ]:
clip_model = SentenceTransformer('clip-ViT-B-32')
#512

In [12]:
def embed_image(path: str) -> np.ndarray:
    img = Image.open(path).convert("RGB")
    emb = clip_model.encode([img], convert_to_numpy=True, show_progress_bar=False)
    return emb[0]

def embed_text(text: str) -> np.ndarray:
    emb = clip_model.encode([text], convert_to_numpy=True, show_progress_bar=False)
    return emb[0]

In [18]:

doc = fitz.open(PDF_PATH)
for i in doc:
    print(i.get_text("blocks"))
    
    

[(124.31300354003906, 72.89631652832031, 487.8945007324219, 112.74653625488281, 'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\n', 0, 0), (211.48800659179688, 148.8298797607422, 399.89349365234375, 166.04527282714844, 'Attention Is All You Need\n', 1, 0), (116.68099975585938, 233.53939819335938, 216.05787658691406, 267.1680908203125, 'Ashish Vaswani∗\nGoogle Brain\navaswani@google.com\n', 2, 0), (230.69200134277344, 233.53939819335938, 309.1475830078125, 267.1680908203125, 'Noam Shazeer∗\nGoogle Brain\nnoam@google.com\n', 3, 0), (323.7870178222656, 233.53939819335938, 407.4730529785156, 267.1680908203125, 'Niki Parmar∗\nGoogle Research\nnikip@google.com\n', 4, 0), (422.11199951171875, 233.53939819335938, 497.210693359375, 267.1680908203125, 'Jakob Uszkoreit∗\nGoogle Research\nusz@google.com\n', 5, 0), (126.88200378417969, 283.5364074707031, 210.5677947998047

In [15]:
doc.metadata

{'format': 'PDF 1.5',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': 'LaTeX with hyperref',
 'producer': 'pdfTeX-1.40.25',
 'creationDate': 'D:20240410211143Z',
 'modDate': 'D:20240410211143Z',
 'trapped': '',
 'encryption': None}

In [19]:
for i in doc:
    print(i.get_images())

[]
[]
[(128, 175, 1520, 2239, 8, 'DeviceRGB', '', 'Im1', 'FlateDecode')]
[(182, 200, 445, 884, 8, 'DeviceRGB', '', 'Im2', 'FlateDecode'), (183, 201, 835, 1282, 8, 'DeviceRGB', '', 'Im3', 'FlateDecode')]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [20]:
image_records: List[Dict[str, Any]] = []

print("Extracting images and captions...")
for page_index, page in enumerate(doc, start=1):
    blocks = page.get_text("blocks")  # (x0, y0, x1, y1, text, ...)
    text_blocks = [b for b in blocks if b[4].strip()]  # keep non-empty text blocks

    images = page.get_images(full=True)
    print(f"Page {page_index}: found {len(images)} images")

    for img_index, img in enumerate(images, start=1):
        xref = img[0]

        # Where is this image on the page?
        rects = page.get_image_rects(xref)
        if not rects:
            continue
        rect = rects[0]

        # Heuristic: caption = closest text block BELOW the image
        below_blocks = [b for b in text_blocks if b[1] >= rect.y1]
        candidates = below_blocks if below_blocks else text_blocks

        if candidates:
            closest_block = min(candidates, key=lambda b: abs(b[1] - rect.y1))
            caption_text = closest_block[4].strip().replace("\n", " ")
        else:
            caption_text = ""

        # Extract image binary and save to file
        img_data = doc.extract_image(xref)
        img_bytes = img_data["image"]
        img_ext = img_data.get("ext", "png")

        img_filename = f"page_{page_index:02d}_img_{img_index}.{img_ext}"
        img_path = os.path.join(FIGURES_DIR, img_filename)

        with open(img_path, "wb") as f:
            f.write(img_bytes)

        image_records.append({
            "id": f"p{page_index}_img{img_index}",
            "page_number": page_index,
            "bbox_x0": float(rect.x0),
            "bbox_y0": float(rect.y0),
            "bbox_x1": float(rect.x1),
            "bbox_y1": float(rect.y1),
            "caption": caption_text,
            "image_path": img_path,
        })

print(f"Total extracted images: {len(image_records)}")

Extracting images and captions...
Page 1: found 0 images
Page 2: found 0 images
Page 3: found 1 images
Page 4: found 2 images
Page 5: found 0 images
Page 6: found 0 images
Page 7: found 0 images
Page 8: found 0 images
Page 9: found 0 images
Page 10: found 0 images
Page 11: found 0 images
Page 12: found 0 images
Page 13: found 0 images
Page 14: found 0 images
Page 15: found 0 images
Total extracted images: 3


In [21]:
text_records: List[Dict[str, Any]] = []

print("Extracting text chunks...")
MIN_CHARS = 80  # ignore very tiny text fragments

for page_index, page in enumerate(doc, start=1):
    blocks = page.get_text("blocks")  # (x0, y0, x1, y1, text, ...)
    for block_idx, b in enumerate(blocks):
        text = b[4].strip()
        if len(text) < MIN_CHARS:
            continue

        x0, y0, x1, y1 = b[0], b[1], b[2], b[3]

        text_records.append({
            "id": f"p{page_index}_block{block_idx}",
            "page_number": page_index,
            "bbox_x0": float(x0),
            "bbox_y0": float(y0),
            "bbox_x1": float(x1),
            "bbox_y1": float(y1),
            "text": text,
        })

print(f"Total extracted text chunks: {len(text_records)}")

Extracting text chunks...
Total extracted text chunks: 123


In [22]:
text_records

[{'id': 'p1_block0',
  'page_number': 1,
  'bbox_x0': 124.31300354003906,
  'bbox_y0': 72.89631652832031,
  'bbox_x1': 487.8945007324219,
  'bbox_y1': 112.74653625488281,
  'text': 'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.'},
 {'id': 'p1_block11',
  'page_number': 1,
  'bbox_x0': 143.5570068359375,
  'bbox_y0': 413.33428955078125,
  'bbox_x1': 469.78717041015625,
  'bbox_y1': 576.0990600585938,
  'text': 'The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these

In [23]:
image_records

[{'id': 'p3_img1',
  'page_number': 3,
  'bbox_x0': 196.5590057373047,
  'bbox_y0': 72.00198364257812,
  'bbox_x1': 415.43902587890625,
  'bbox_y1': 394.4179992675781,
  'caption': 'Figure 1: The Transformer - model architecture.',
  'image_path': 'figures\\page_03_img_1.png'},
 {'id': 'p4_img1',
  'page_number': 4,
  'bbox_x0': 174.9600067138672,
  'bbox_y0': 94.02497863769531,
  'bbox_x1': 239.04000854492188,
  'bbox_y1': 221.32098388671875,
  'caption': 'Figure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several attention layers running in parallel.',
  'image_path': 'figures\\page_04_img_1.png'},
 {'id': 'p4_img2',
  'page_number': 4,
  'bbox_x0': 346.7720031738281,
  'bbox_y0': 82.68698120117188,
  'bbox_x1': 467.0119934082031,
  'bbox_y1': 267.29498291015625,
  'caption': 'Figure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several attention layers running in parallel.',
  'image_path': 'figures\\page_04_img

In [25]:
fig_collection = client.get_or_create_collection(
    name=FIG_COLLECTION_NAME,
    metadata={"type": "image", "source": "attention_pdf"},
)

text_collection = client.get_or_create_collection(
    name=TEXT_COLLECTION_NAME,
    metadata={"type": "text", "source": "attention_pdf"},
)


In [26]:
fig_ids = []
fig_embeddings = []
fig_metadatas = []
fig_documents = []

for rec in image_records:
    img_emb = embed_image(rec["image_path"])

    fig_ids.append(rec["id"])
    fig_embeddings.append(img_emb.tolist())
    fig_metadatas.append({
        "page_number": int(rec["page_number"]),
        "bbox_x0": rec["bbox_x0"],
        "bbox_y0": rec["bbox_y0"],
        "bbox_x1": rec["bbox_x1"],
        "bbox_y1": rec["bbox_y1"],
        "image_path": rec["image_path"],
        "caption": rec["caption"],
    })
    fig_documents.append(
        f"Image from 'Attention Is All You Need' on page {rec['page_number']}. "
        f"Caption: {rec['caption']}"
    )

if fig_ids:
    fig_collection.add(
        ids=fig_ids,
        embeddings=fig_embeddings,
        metadatas=fig_metadatas,
        documents=fig_documents,
    )

In [27]:
txt_ids = []
txt_embeddings = []
txt_metadatas = []
txt_documents = []

for rec in text_records:
    emb = embed_text(rec["text"])

    txt_ids.append(rec["id"])
    txt_embeddings.append(emb.tolist())
    txt_metadatas.append({
        "page_number": int(rec["page_number"]),
        "bbox_x0": rec["bbox_x0"],
        "bbox_y0": rec["bbox_y0"],
        "bbox_x1": rec["bbox_x1"],
        "bbox_y1": rec["bbox_y1"],
    })
    txt_documents.append(rec["text"])

if txt_ids:
    text_collection.add(
        ids=txt_ids,
        embeddings=txt_embeddings,
        metadatas=txt_metadatas,
        documents=txt_documents,
    )

In [28]:
def answer_query(
    query: str,
    top_k_text: int = 5,
    top_k_img: int = 5,
    top_k_overall: int = 8,
) -> List[Dict[str, Any]]:
    """
    Search both text chunks and images in Chroma Cloud,
    merge results by distance, and return/print top hits.
    """
    q_emb = embed_text(query)

    combined: List[Dict[str, Any]] = []

    # --- Text search ---
    text_results = text_collection.query(
        query_embeddings=[q_emb.tolist()],
        n_results=top_k_text,
        include=["metadatas", "documents", "distances"],
    )

    for meta, doc, dist in zip(
        text_results["metadatas"][0],
        text_results["documents"][0],
        text_results["distances"][0],
    ):
        combined.append({
            "type": "text",
            "distance": float(dist),
            "page_number": meta["page_number"],
            "bbox": (
                meta["bbox_x0"],
                meta["bbox_y0"],
                meta["bbox_x1"],
                meta["bbox_y1"],
            ),
            "content": doc,
        })

    # --- Image search ---
    fig_results = fig_collection.query(
        query_embeddings=[q_emb.tolist()],
        n_results=top_k_img,
        include=["metadatas", "documents", "distances"],
    )

    for meta, doc, dist in zip(
        fig_results["metadatas"][0],
        fig_results["documents"][0],
        fig_results["distances"][0],
    ):
        combined.append({
            "type": "image",
            "distance": float(dist),
            "page_number": meta["page_number"],
            "image_path": meta["image_path"],
            "caption": meta.get("caption", ""),
            "content": doc,
        })

    # Sort by similarity (lower distance = more similar)
    combined.sort(key=lambda x: x["distance"])
    combined = combined[:top_k_overall]

    # Pretty-print
    print(f"\n=== Query: {query} ===")
    for rank, item in enumerate(combined, start=1):
        print(f"\nRank {rank} | type: {item['type']} | distance: {item['distance']:.4f}")
        print("  Page:", item["page_number"])
        if item["type"] == "text":
            preview = item["content"].replace("\n", " ")
            if len(preview) > 260:
                preview = preview[:260] + "..."
            print("  Text:", preview)
        else:
            print("  Image path:", item["image_path"])
            print("  Caption   :", item.get("caption", ""))

    return combined

In [32]:
answer_query("transfermer model architecture",top_k_text=2, top_k_img=1, top_k_overall=3)


=== Query: transfermer model architecture ===

Rank 1 | type: text | distance: 32.6148
  Page: 10
  Text: For translation tasks, the Transformer can be trained significantly faster than architectures based on recurrent or convolutional layers. On both WMT 2014 English-to-German and WMT 2014 English-to-French translation tasks, we achieve a new state of the art. In...

Rank 2 | type: text | distance: 35.0929
  Page: 2
  Text: In this work we propose the Transformer, a model architecture eschewing recurrence and instead relying entirely on an attention mechanism to draw global dependencies between input and output. The Transformer allows for significantly more parallelization and ca...

Rank 3 | type: image | distance: 138.0081
  Page: 4
  Image path: figures\page_04_img_2.png
  Caption   : Figure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several attention layers running in parallel.


[{'type': 'text',
  'distance': 32.614773,
  'page_number': 10,
  'bbox': (108.0, 427.23828125, 503.99761962890625, 470.0030822753906),
  'content': 'For translation tasks, the Transformer can be trained significantly faster than architectures based\non recurrent or convolutional layers. On both WMT 2014 English-to-German and WMT 2014\nEnglish-to-French translation tasks, we achieve a new state of the art. In the former task our best\nmodel outperforms even all previously reported ensembles.'},
 {'type': 'text',
  'distance': 35.092937,
  'page_number': 2,
  'bbox': (107.69100189208984,
   300.9082946777344,
   505.73870849609375,
   343.6730651855469),
  'content': 'In this work we propose the Transformer, a model architecture eschewing recurrence and instead\nrelying entirely on an attention mechanism to draw global dependencies between input and output.\nThe Transformer allows for significantly more parallelization and can reach a new state of the art in\ntranslation quality after b